In [ ]:
!pip install -U aifactory
!pip install -U gdown
!pip install tfts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=87abef9e2515cff4edcec9bca5847921f297141318711952777ad636115647a2
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 kB 13.9 MB/s

In [ ]:
# import gdown

# gdown.download(id='1ayOEPC3imrnhVO5RAgUp2rCbka-0vgUO', output='train_mean.pickle', quiet=False)
# gdown.download(id='1hr1Gh-Bhm7afVj2-6nD_z8AuV3s7LAEd', output='train_std.pickle', quiet=False)
# gdown.download(id='1m8g0iJKLUq24RNyloCT5UzAkZoO9-NOG', output='model_weights.h5', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1ayOEPC3imrnhVO5RAgUp2rCbka-0vgUO
To: /content/train_mean.pickle
100%|██████████| 833/833 [00:00<00:00, 2.76MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hr1Gh-Bhm7afVj2-6nD_z8AuV3s7LAEd
To: /content/train_std.pickle
100%|██████████| 833/833 [00:00<00:00, 851kB/s]
Downloading...
From: https://drive.google.com/uc?id=1m8g0iJKLUq24RNyloCT5UzAkZoO9-NOG
To: /content/model_weights.h5
100%|██████████| 165k/165k [00:00<00:00, 3.02MB/s]


'model_weights.h5'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tfts
import pickle
import random
from tfts import AutoModel, KerasTrainer
from tensorflow.keras.layers import Dense, Input

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


def load_model():
    # 모델 아키텍쳐
    inputs = Input([6, 6]) #  time_step, columns = 6,6
    backbone = AutoModel("WaveNet")
    outputs = backbone(inputs)
    outputs = Dense(1)(outputs)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

    # 모델 가중치 로드
    model.load_weights('model_weights.h5')

    return model


# 예측 함수는 AIF에서 제공하는 X_test의 입력 정보와, 메모리에 올라간 사전 학습 모델을 입력 받습니다.
def predict(X_test, model):
    sep_x = X_test.reshape(-1,6).copy()

    # train 데이터의 mean, std 정보 (정규화 수행하기 위해)
    with open("train_mean.pickle","rb") as fr:
        train_mean = pickle.load(fr)
    with open("train_std.pickle","rb") as fr:
        train_std = pickle.load(fr)
    
    test_x = (sep_x - train_mean.values[1:]) / train_std.values[1:]
    test_x = test_x.reshape(-1,6,6)


    pred = model.predict(test_x)
    pred = pred.reshape(-1)

    pred = pred * train_std.values[0] + train_mean.values[0]
    return pred

def submit():
    return load_model, predict



import aifactory.score as aif

if __name__ == "__main__":
    aif.submit(model_name="MyFirstSubmission",
               key='98a9b6f7-bc81-4e4f-9ebe-272fae734108',
               func=submit # submit function
               )

file : fileId=18pJZsGXjnnRP3GUQTIV3Y-AN9qrRXve-
Running on CoLab
google colab


Downloading...
From: https://drive.google.com/uc?id=18pJZsGXjnnRP3GUQTIV3Y-AN9qrRXve-
To: /content/task.ipynb
100%|██████████| 20.5k/20.5k [00:00<00:00, 39.2MB/s]


request id : 191 processing...
score = 0.6121525
